# 05 Active vs Benchmark
Evaluate strategies relative to XLV using active returns, IR, and overlays.

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data.etf_loader import load_clean_prices
from run_strategies import run_rotation_strategy, run_regime_strategy
from src.backtest.engine import run_backtest
from src.analysis.metrics import (
    compute_cagr, compute_annual_vol, compute_sharpe, compute_max_drawdown,
    compute_information_ratio, compute_correlation, compute_blended_returns,
)

%matplotlib inline


### Load data and run strategies
Re-use pipeline from run_strategies.py (10 bps costs).

In [ ]:
prices = load_clean_prices().dropna()
rotation_bt = run_rotation_strategy(prices, tc_bps=10.0)
regime_bt, risk_on_frac = run_regime_strategy(prices, tc_bps=10.0, start=None, end=None)
print(f'Risk-on fraction (regime): {risk_on_frac:.1%}')
xlv = prices['XLV'].pct_change().fillna(0.0)
xlv_equity = (1 + xlv).cumprod()
hc_cols = [c for c in ['XBI','XPH','IHF','IHI','XLV'] if c in prices.columns]
month_ends = prices.resample('ME').last().index
ew_monthly = pd.DataFrame(1/len(hc_cols), index=month_ends, columns=hc_cols)
ew_daily = ew_monthly.reindex(prices.index, method='ffill').fillna(0.0)
ew_bt = run_backtest(prices[hc_cols], ew_daily, transaction_cost_bps=10.0)


### Active returns vs XLV and IR / correlation


In [ ]:
rot_active = rotation_bt.daily_returns - xlv
reg_active = regime_bt.daily_returns - xlv
rot_ir = compute_information_ratio(rotation_bt.daily_returns, xlv)
reg_ir = compute_information_ratio(regime_bt.daily_returns, xlv)
rot_corr = compute_correlation(rotation_bt.daily_returns, xlv)
reg_corr = compute_correlation(regime_bt.daily_returns, xlv)
summary = pd.DataFrame([
    {"strategy": "Rotation", "CAGR": compute_cagr(rotation_bt.daily_returns), "Vol": compute_annual_vol(rotation_bt.daily_returns), "Sharpe": compute_sharpe(rotation_bt.daily_returns), "MaxDD": compute_max_drawdown(rotation_bt.equity_curve), "IR_vs_XLV": rot_ir, "Corr_vs_XLV": rot_corr},
    {"strategy": "Regime LS", "CAGR": compute_cagr(regime_bt.daily_returns), "Vol": compute_annual_vol(regime_bt.daily_returns), "Sharpe": compute_sharpe(regime_bt.daily_returns), "MaxDD": compute_max_drawdown(regime_bt.equity_curve), "IR_vs_XLV": reg_ir, "Corr_vs_XLV": reg_corr},
    {"strategy": "XLV", "CAGR": compute_cagr(xlv), "Vol": compute_annual_vol(xlv), "Sharpe": compute_sharpe(xlv), "MaxDD": compute_max_drawdown(xlv_equity), "IR_vs_XLV": float('nan'), "Corr_vs_XLV": 1.0},
])
summary[['strategy','CAGR','Vol','Sharpe','MaxDD','IR_vs_XLV','Corr_vs_XLV']]


### Blended overlays: XLV + LS
Test small overlays of LS on top of XLV.

In [ ]:
blend_10 = compute_blended_returns(xlv, regime_bt.daily_returns, overlay_weight=0.10)
blend_20 = compute_blended_returns(xlv, regime_bt.daily_returns, overlay_weight=0.20)
blends = pd.DataFrame([
    {"strategy": "XLV+10% LS", "CAGR": compute_cagr(blend_10), "Vol": compute_annual_vol(blend_10), "Sharpe": compute_sharpe(blend_10), "MaxDD": compute_max_drawdown((1+blend_10).cumprod())},
    {"strategy": "XLV+20% LS", "CAGR": compute_cagr(blend_20), "Vol": compute_annual_vol(blend_20), "Sharpe": compute_sharpe(blend_20), "MaxDD": compute_max_drawdown((1+blend_20).cumprod())},
    {"strategy": "XLV", "CAGR": compute_cagr(xlv), "Vol": compute_annual_vol(xlv), "Sharpe": compute_sharpe(xlv), "MaxDD": compute_max_drawdown(xlv_equity)},
])
blends


### Commentary
- IR > 0 suggests incremental value vs XLV; IR ~ 0 means limited edge.
- Low correlation to XLV can justify a small overlay even if standalone Sharpe is modest.
- Blends that improve Sharpe or reduce drawdown vs pure XLV indicate potential PM use.